In [22]:
import os
import sys
from pathlib import Path

class Config:
    project_dir = Path(os.getcwd())
    logs_dir = project_dir / "logs"
    results_dir = project_dir / "results"
    data_dir = Path("/home/nikita/Data/drw-crypto-market-prediction")
    bitcoin_dir = Path("/home/nikita/Data/Bitcoin1min")
    
print(Config.project_dir)
print(Config.logs_dir)
print(Config.results_dir)
print(Config.data_dir)



from typing import Tuple, List, Union, Any, Optional, Dict, Literal, Callable
import time
import json
import os
import sys
import gc


import numpy as np
import pandas as pd
import seaborn as sns






#set pandas display options
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.width', 1000)
# Set numpy print options
np.set_printoptions(precision=3, suppress=True, linewidth=1000)

/home/nikita/Code/kaggle
/home/nikita/Code/kaggle/logs
/home/nikita/Code/kaggle/results
/home/nikita/Data/drw-crypto-market-prediction


# load data

In [27]:
train = pd.read_parquet(Config.data_dir / "train.parquet",
                        columns= ["bid_qty", "ask_qty", "buy_qty", "sell_qty", "volume", "label"])
test = pd.read_parquet(Config.data_dir / "test.parquet",
                          columns= ["bid_qty", "ask_qty", "buy_qty", "sell_qty", "volume"])

In [29]:
train

,bid_qty,ask_qty,buy_qty,sell_qty,volume,label
timestamp,,,,,,
2023-03-01 00:00:00,15.283,8.425,176.405,44.984,221.389,0.562539
2023-03-01 00:01:00,38.590,2.336,525.846,321.950,847.796,0.533686
2023-03-01 00:02:00,0.442,60.250,159.227,136.369,295.596,0.546505
2023-03-01 00:03:00,4.865,21.016,335.742,124.963,460.705,0.357703
2023-03-01 00:04:00,27.158,3.451,98.411,44.407,142.818,0.362452
...,...,...,...,...,...,...
2024-02-29 23:55:00,4.163,6.805,39.037,55.351,94.388,0.396289
2024-02-29 23:56:00,2.290,4.058,110.201,67.171,177.372,0.328993
2024-02-29 23:57:00,5.237,3.640,70.499,30.753,101.252,0.189909


In [10]:
len(test)

538150

In [8]:
train

,bid_qty,ask_qty,buy_qty,sell_qty,volume
timestamp,,,,,
2023-03-01 00:00:00,15.283,8.425,176.405,44.984,221.389
2023-03-01 00:01:00,38.590,2.336,525.846,321.950,847.796
2023-03-01 00:02:00,0.442,60.250,159.227,136.369,295.596
2023-03-01 00:03:00,4.865,21.016,335.742,124.963,460.705
2023-03-01 00:04:00,27.158,3.451,98.411,44.407,142.818
...,...,...,...,...,...
2024-02-29 23:55:00,4.163,6.805,39.037,55.351,94.388
2024-02-29 23:56:00,2.290,4.058,110.201,67.171,177.372
2024-02-29 23:57:00,5.237,3.640,70.499,30.753,101.252


In [59]:
# train.loc[::24*60]


In [73]:
train

,bid_qty,ask_qty,buy_qty,sell_qty,volume,label
timestamp,,,,,,
2023-03-01 00:00:00,15.283,8.425,176.405,44.984,221.389,0.562539
2023-03-01 00:01:00,38.590,2.336,525.846,321.950,847.796,0.533686
2023-03-01 00:02:00,0.442,60.250,159.227,136.369,295.596,0.546505
2023-03-01 00:03:00,4.865,21.016,335.742,124.963,460.705,0.357703
2023-03-01 00:04:00,27.158,3.451,98.411,44.407,142.818,0.362452
...,...,...,...,...,...,...
2024-02-29 23:55:00,4.163,6.805,39.037,55.351,94.388,0.396289
2024-02-29 23:56:00,2.290,4.058,110.201,67.171,177.372,0.328993
2024-02-29 23:57:00,5.237,3.640,70.499,30.753,101.252,0.189909


In [74]:
train.iloc[0]

bid_qty      15.283000
ask_qty       8.425000
buy_qty     176.405000
sell_qty     44.984000
volume      221.389000
label         0.562539
Name: 2023-03-01 00:00:00, dtype: float64

In [75]:
train.iloc[-1]

bid_qty       3.925000
ask_qty       3.865000
buy_qty      86.585000
sell_qty    217.137000
volume      303.722000
label         0.731542
Name: 2024-02-29 23:59:00, dtype: float64

# load binance data

In [71]:
vendors = ["Binance", "Bitfinex", "BitMEX", "Coinbase", "Combined_Index", "KuCoin", "OKX"]
bitcoin = pd.read_csv(Config.bitcoin_dir / "BTCUSD_1m_KuCoin.csv",
                      parse_dates=["Open time"],
                      index_col="Open time",
                      )["2023-03-01":]

In [72]:
bitcoin.head()

,Open,Close,High,Low,Volume,Amount
Open time,,,,,,
2023-03-01 00:00:00,23141.9,23144.1,23150.0,23134.1,6.493077,150270.462109
2023-03-01 00:01:00,23144.1,23143.4,23150.0,23143.4,2.607217,60349.442499
2023-03-01 00:02:00,23143.5,23155.5,23158.4,23143.5,1.681010,38915.873951
2023-03-01 00:03:00,23155.5,23158.5,23159.6,23155.5,1.483790,34362.260198
2023-03-01 00:04:00,23158.5,23175.5,23175.7,23158.5,9.880489,228932.659059
